In [1]:
import pandas as pd
import numpy as np
import os

In [6]:
model_path="/home/sumbhati/ingenious/LM-pretraining/models/regular_bert_13_06_2022_18:22:00/step_250000"
data=pd.DataFrame(columns=["cola", "mrpc", "rte", "stsb", "sst2", "mnli-m", "mnli-mm", "qnli", "qqp"], index=[f"glue_run_{i}" for i in range(1, 21)])
for run in range(1, 21):
    if run>5:
        tasks=["cola", "mrpc", "rte", "stsb"]
    else:
        tasks=["cola", "mrpc", "rte", "stsb", "sst2", "mnli", "qnli", "qqp"]
    l=[]
    for task in tasks:
        with open(f"{model_path}/glue_run_{run}/{task}.log", "r") as f:
            lines=f.readlines()
            if task=="mnli":
                l.append(100*float(lines[-2].split()[-1].rstrip().rstrip('}')))
            l.append(100*float(lines[-1].split()[-1].rstrip().rstrip('}')))
    if run>5:
        l.extend([np.nan]*5)
    data.loc[f"glue_run_{run}"]=l
data.loc["average"]=data.mean(axis=0, skipna=True)
data.loc["stddev"]=data.std(axis=0, skipna=True)
data.loc["decrement_avg"]=[np.nan for i in range(9)]
data.loc["maximum"]=data.max(axis=0, skipna=True)
data.loc["decrement_max"]=[np.nan for i in range(9)]
data.to_csv(f"{model_path}/glue_results.csv")

In [7]:
data

,cola,mrpc,rte,stsb,sst2,mnli-m,mnli-mm,qnli,qqp
glue_run_1,54.905472,90.456432,66.666667,89.350008,91.183036,82.518263,83.380682,90.134448,87.544413
glue_run_2,48.717238,89.349931,63.802083,88.572571,91.183036,82.924107,83.127029,90.207122,87.753945
glue_run_3,46.781964,91.036415,67.96875,89.027584,90.848214,82.426948,83.258929,90.406977,87.568256
glue_run_4,52.999022,88.765603,65.104167,88.685992,91.183036,83.258929,83.005276,90.188953,87.559872
glue_run_5,49.060244,90.809328,67.447917,88.642338,91.629464,82.518263,82.99513,89.934593,87.368315
glue_run_6,48.662515,90.534979,66.927083,88.782421,NaN,NaN,NaN,NaN,NaN
glue_run_7,51.099499,90.809328,67.708333,89.067261,NaN,NaN,NaN,NaN,NaN
glue_run_8,54.876014,90.633609,65.364583,89.273121,NaN,NaN,NaN,NaN,NaN
glue_run_9,49.039385,91.655267,67.708333,89.055844,NaN,NaN,NaN,NaN,NaN
glue_run_10,51.566309,89.260809,64.84375,89.111984,NaN,NaN,NaN,NaN,NaN
